この記事は[Julia Advent Calendar 2017](https://qiita.com/advent-calendar/2017/julialang)の17日目の記事です．
普段はpythonばかり書いていて，juliaは最近文法覚えてきたかなレベルなので色々許してください．

## 式or擬似コードに可能な限り近いプログラム

- 自分で解いた数式を実装するとき
- 論文に書いてある擬似コードを実装するとき

式or擬似コードに可能な限り近いプログラムを書くようにしている．
ここで"可能な限り近い"とは，関数の名前とかを合わせるとかだけでなく，$\alpha$などunicode文字をバンバン使うことを意味する．このようなプログラムを書くことで，

- デバッグがしやすい
- 頭を使わなくてもプログラミングできる

という利点がある．

例えば[No-U-Turn Sampler Hoffman+, 2011](https://arxiv.org/abs/1111.4246)の擬似コード(の一部，論文より引用)は

[f:id:ksknw:20171130224045p:plain]

これに対して書いたpythonコードは以下．

```python
def BuildTree(θ, r, u, v, j, ε):
    if j == 0:
        θd, rd = Leapfrog(x, θ, r, v * ε)
        if np.log(u) <= (L(*θd) - 0.5 * np.dot(rd, rd)):
            Cd_ = [[θd, rd]]
        else:
            Cd_ = []
        sd = int(np.log(u) < (Δ_max + L(*θd) - 0.5 * np.dot(rd, rd)))
        return θd, rd, θd, rd, Cd_, sd
    else:
        θ_minus, r_minus, θ_plus, r_plus, Cd_, sd = BuildTree(θ, r, u, v, j - 1, ε)
        if v == -1:
            θ_minus, r_minus, _, _, Cdd_, sdd = BuildTree(θ_minus, r_minus, u, v, j - 1, ε)
        else:
            _, _, θ_plus, r_plus, Cdd_, sdd = BuildTree(θ_plus, r_plus, u, v, j - 1, ε)
        sd = sdd * sd * int((np.dot(θ_plus - θ_minus, r_minus) >= 0) and (np.dot(θ_plus - θ_minus, r_plus) >= 0))
        Cd_.extend(Cdd_)

        return θ_minus, r_minus, θ_plus, r_plus, Cd_, sd
```

pythonではある程度unicodeを変数に使うことができ，例えばギリシャ文字などは全て思うように書ける．
しかし，一部の例えば$\theta ^+$や$\nabla$などの記号は使うことができないため，微妙に見難い感じになってしまっていた．
(あとpythonはまじで遅い)

探してみるとjuliaで同じことをしている人がすでにいて，こちらのほうがだいぶ良さそうだった．

[http://bicycle1885.hatenablog.com/entry/2014/12/05/011256:embed:cite]

juliaでは↑であげたような文字に加えて$\hat$みたいな修飾文字も[変数名に使えるらしい](https://docs.julialang.org/en/release-0.4/manual/unicode-input/)．
さらに不等号の$\le$とかが定義されていて使えるらしい．
juliaすごい．あとなんか速いらしい．pythonには実装されていない()多重入れ子なfor文も使っていいらしい．

ということで以下では練習がてら，これまでpythonで実装したコードをjuliaで書きなおしてみて，数式/擬似コードの再現度と実行速度を比較する．

NUTSはもういいかなって感じなので

- Dynamic Time Warping
- Stochastic Block Model

について実装する．

## Dynamic Time Warping
[http://ksknw.hatenablog.com/entry/2017/03/26/234048:embed:cite]

Dynamic Time Warping (DTW) は，２つの連続値系列データのいい感じの距離を求めるアルゴリズム．
動的計画法で解く．

(DTW自体の論文ではないけど) [A global averaging method for dynamic time warping, with applications to clustering](http://www.francois-petitjean.com/Research/Petitjean2011-PR.pdf)
の擬似コードを参考にしてプログラムを書く．
擬似コードは以下．

[f:id:ksknw:20171130230716p:plain]

[f:id:ksknw:20171130230720p:plain]

| python | julia |
|--------|-------|
|<script src="https://gist.github.com/kskkwn/3faac8748640744cb7ccf3c4254bc195.js"></script>  |<script src="https://gist.github.com/kskkwn/3faac8748640744cb7ccf3c4254bc195.js"></script>
|


## Stochastic Block Model

In [2]:
using Einsum
using DataFrames
using CSV
using StatsBase

In [3]:
K = 8
α = 6
a₀ = b₀ = 0.5

α₁ = transpose(ones(K) * α) # =α₂ 
logΓ = lgamma

lgamma (generic function with 13 methods)

In [4]:
function onehot(i, K)
    ret = zeros(K)
    ret[i] = 1
    return ret
end

function m(x)
    return sum(x,1)
end

m (generic function with 1 method)

In [4]:
# 𝕀 # \BbbI 

LoadError: UndefVarError: 𝕀 not defined

In [36]:
function update_z₁(X, 𝕀z₁, 𝕀z₂)
    N₁, N₂ = size(X)
    m₁ = m(𝕀z₁)
    
    for i in 1:N₁
        @einsum n⁺[k,l] := X[i,j] * 𝕀z₁[i,k] * 𝕀z₂[j,l]
        @einsum n⁻[k,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₁[i,k] * 𝕀z₂[j,l]
    
        m̂₁ = m(𝕀z₁) - transpose(𝕀z₁[i,:])
        @einsum Σ⁺x𝕀z₂[i,l] := X[i,j] * 𝕀z₂[j,l]
        @einsum Σ⁻x𝕀z₂[i,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₂[j,l]
        @einsum n̂⁺[k,l] := n⁺[k,l] - 𝕀z₁[i,k] * Σ⁺x𝕀z₂[i,l]
        @einsum n̂⁻[k,l] := n⁻[k,l] - 𝕀z₁[i,k] * Σ⁻x𝕀z₂[i,l]
        
        α̂₁ = α₁ + m̂₁
        â = a₀ + n̂⁺
        b̂ = b₀ + n̂⁻
        
        temp⁺ = zeros(â)
        temp⁻ = zeros(â)
        temp = zeros(â)
        for j in 1:size(temp⁺)[1]
            temp⁺[j,:] = Σ⁺x𝕀z₂[i,:]
            temp⁻[j,:] = Σ⁻x𝕀z₂[i,:]
            temp[j,:] = sum(𝕀z₂,1)
        end
        
        @einsum p_z₁[k,l] := exp(logΓ(â + b̂)-logΓ(â)-logΓ(b̂) 
            + logΓ(â+temp⁺)+logΓ(b̂+temp⁻)-logΓ(â+b̂+temp))[k,l]
        p_z₁ = α̂₁ .* transpose(prod(p_z₁, 2))
        p_z₁ /= sum(p_z₁)
        
        𝕀z₁[i,:] = onehot(sample(1:K, Weights(p_z₁[:])), K)
    end
    return 𝕀z₁
end

update_z₁ (generic function with 1 method)

In [13]:
function update_z₁(X, 𝕀z₁, 𝕀z₂)
    N₁, N₂ = size(X)
    m₁ = m(𝕀z₁)
    
    for i in 1:N₁
        m̂₁ = m(𝕀z₁) - transpose(𝕀z₁[i,:])      
        n⁺ = zeros(K, K)
        n⁻ = zeros(K, K)

        Σ⁺x𝕀z₂ = zeros(K)
        Σ⁻x𝕀z₂ = zeros(K)
        for l in 1:K
            for j in 1:N₂
                Σ⁺x𝕀z₂[l] += X[i,j] * 𝕀z₂[j,l]
                Σ⁻x𝕀z₂[l] += (1 - X[i,j]) * 𝕀z₂[j,l]
                for k in 1:K
                    n⁺[k,l] += 𝕀z₁[i,k] * X[i,j] *  𝕀z₂[j,l]
                    n⁻[k,l] += (1 - X[i,j]) * 𝕀z₁[i,k] * 𝕀z₂[j,l]
                end
            end
        end

        n̂⁺ = zeros(K,K)
        n̂⁻ = zeros(K,K) 
        for k in 1:K
            for l in 1:K
                n̂⁺[k,l] += n⁺[k,l] - 𝕀z₁[i,k] * Σ⁺x𝕀z₂[l]
                n̂⁻[k,l] += n⁻[k,l] - 𝕀z₁[i,k] * Σ⁻x𝕀z₂[l]
            end
        end
        α̂₁ = α₁ + m̂₁
        â = a₀ + n̂⁺
        b̂ = b₀ + n̂⁻
        
        p_z₁ = α̂₁ 
        for k in 1:K
            for l in 1:K
                p_z₁[k] *= exp(logΓ(â[k,l] + b̂[k,l])-logΓ(â[k,l])-logΓ(b̂[k,l]) \
                    + logΓ(â[k,l]+Σ⁺x𝕀z₂[l])+logΓ(b̂[k,l]+Σ⁻x𝕀z₂[l])-logΓ(â[k,l]+b̂[k,l]+sum(𝕀z₂,1)[l]))
            end
        end
        p_z₁ /= sum(p_z₁)
        
        𝕀z₁[i,:] = onehot(sample(1:K, Weights(p_z₁[:])), K)
    end
    return 𝕀z₁
end

update_z₁ (generic function with 1 method)

In [6]:
data = readtable("./bi_data.csv")
X = hcat(data.columns...)

120×120 DataArrays.DataArray{Int64,2}:
 0  0  1  0  0  0  0  1  0  1  1  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  1  0  1  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  1  1  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  

In [7]:
𝕀z₁ = zeros(size(X)[1],K)
𝕀z₁[:,1] = 1
𝕀z₂ = zeros(size(X)[1],K)
𝕀z₂[:,1] = 1

1

In [22]:
𝕀z₁ = update_z₁(X, 𝕀z₁, 𝕀z₂)
𝕀z₂ = update_z₁(transpose(X), 𝕀z₂, 𝕀z₁)

120×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 ⋮                        ⋮            
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  

In [21]:
mxval, mxindx = findmax(𝕀z₂, 2)
for i in ind2sub(size(𝕀z₂), vec(mxindx))[2]
    print(i)
end

322447478825664212147437652422468834444722784825321734274164332453182257285516742713125317881145784831471727124227425134

関数直した時に，その関数を読んでる関数ももう一度実行しないといけない．